In [1]:
import os
os.chdir("../")

In [2]:
from deepclassifier.constant import *
from deepclassifier.utils import read_yaml , create_directories

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir:Path
    base_model_path:Path
    updated_base_model_path:Path
    params_imgae_size:list
    params_epochs:int
    params_learning_rate:float
    params_include_top:bool
    params_weight:str
    params_classes:int


In [4]:
from deepclassifier.constant import *
from deepclassifier.utils import read_yaml , create_directories

In [5]:
class configurationManager:
    def __init__(self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifact_root])
        
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        create_directories([self.config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            updated_base_model_path = Path(config.updated_base_model_path),
            params_image_size = self.params.IMAGE_SIZE,
            params_weights = self.params.WEIGHTS,
            params_batch_size = self.params.BATCH_SIZE, 
            params_include_top = self.params.INCLUDE_TOP,
            params_epochs = self.params.EPOCHS,
            params_learning_rate = self.params.LEARNING_RATE,
            params_classes = self.params.CLASSES
        )

        return prepare_base_model_config

In [6]:
from deepclassifier.utils import *
from deepclassifier.constant import *


In [7]:
from importlib.resources import path
import os
import tensorflow as tf


class PrepareBaseModel:
    def __init__ (self, config:PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.base_model = tf.keras.applications.vgg16.VGG16(
            weights=self.config.params_weight,
            input_shape=self.config.params_imgae_size,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path,model=self.base_model)

    def _prepare_full_model(model,classes,learnng_rate,freeze_all,freeze_till):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
        
    

    @staticmethod
    def save_model(path:Path , model : tf.keras.Model):
        model.save(path)

In [8]:
from logging import exception


try:
    config = configurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()

except Exception as e:
    raise e


[2022-09-24 08:41:13,808: INFO: common]: yaml file: configs\config.yaml loaded successfully
[2022-09-24 08:41:13,837: INFO: common]: yaml file: params.yaml loaded successfully


BoxKeyError: "'ConfigBox' object has no attribute 'artifact_root'"

In [ ]:
def read_yaml1(path_to_yaml: str):
    """reads yaml file and returns
    Args:
        path_to_yaml (str): path like input
    Raises:
        ValueError: if yaml file is empty
        e: empty file
    Returns:
        ConfigBox: ConfigBox type
    """
    try:
        with open(path_to_yaml) as yaml_file:
            content = yaml.safe_load(yaml_file)
            #logger.info(f"yaml file: {path_to_yaml} loaded successfully")
            return (content)
    except Exception as e:
        raise e

In [ ]:
read_yaml1(Path("configs\config.yaml"))


FileNotFoundError: [Errno 2] No such file or directory: 'configs\\config.yaml'